In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 500)

from sklearn import svm
import re
from shutil import copyfile
import matplotlib.pyplot as plt
import pydicom
from tqdm import tqdm
import nibabel as nib
import dicom2nifti

https://nda.nih.gov/edit_collection.html?QA=false&id=2126

In [2]:
# define text files and path

#### CHANGE FOR SPECIFIC DOWNLOAD PACKAGE ####

# for 2274:
path = '../../physcosis/Package_1187332/'


## for 2126:
# path = '../../physcosis/Package_1187335/'


# should be standard
test_file = 'panss01.txt'
subject_file = 'ndar_subject01.txt'
image_file = 'image03.txt'





In [3]:
# opensfiles using pandas
# drop rows that are fully duplicated from the start --> not need 
ndar_subject = pd.read_csv(os.path.join(path, subject_file), delimiter = '\t')
print(ndar_subject.shape)
ndar_subject = ndar_subject.drop_duplicates()
print(ndar_subject.shape)
test = pd.read_csv(os.path.join(path, test_file), delimiter = '\t')
print(test.shape)
test = test.drop_duplicates()
print(test.shape)
image03 = pd.read_csv(os.path.join(path, image_file), delimiter = '\t')
print(image03.shape)
image03 = image03.drop_duplicates()
print(image03.shape)

(4404, 26)
(4404, 26)
(1779, 42)
(1779, 42)
(9395, 51)
(9395, 51)


## preprocess image03

In [4]:

# shows the different types of images available for the data package
image03.pivot_table(columns = ['image_description'], aggfunc = 'size')

image_description
ADNI Double_TSE SENSE                                                      561
ADNI MPRAGE SENSE                                                           45
ADNI MPRAGE pg7 SENSE                                                      516
AX T2                                                                       16
Axial PD-T2 TSE                                                            118
Axial PD-T2 TSE no angle                                                    73
Axial PD-T2 TSE_ NO ANGULATION                                               4
Axial PD-T2 TSE_NO_ANGLE                                                    26
B1-Calibration Body                                                         98
B1-Calibration Body_COIL                                                    25
B1-Callibration Head                                                        98
B1-Callibration Head_COIL                                                   24
Bsnip Ax PD-T2 TSE NoAng          

In [5]:
types = list(image03['image_description'].unique())


#### CHANGE REGEX FOR SPECIFIC PACKAGE DEPENDING ON WHAT IMAGES ARE NEEDED ####
# for 2274:
regex = re.compile(r'.*MPRAGE.*|.*mprage.*|.*T1.*')
# regex = re.compile(r'.*')


with_mprage = list(filter(regex.search, types))
image03 = image03[image03['image_description'].isin(with_mprage)]
image03 = image03.drop_duplicates()
print(with_mprage)

#### KEEP ONLY WANTED COLUMNS optionally change if more/less are needed ####
print(image03.columns)
print(image03.shape)
image03 = image03[['collection_id', 'image03_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_age', 'image_file', 'image_description']]
image03.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')

['ADNI MPRAGE pg7 SENSE', 'ADNI MPRAGE SENSE', 'WIP ADNI MPRAGE pg7 SENSE', 'SAG MPRAGE', 'T1 MPRAGE', 'T1 MPRAGE Repeat', 'MPRAGE_repeat', 'MPRAGE', 'MPRAGE Repeat', 'MPRAGE_SAG Repeat', 'MPRAGE_SAG', 'Bsnip Sag MPRAGE-like ADNI', 'Sagittal MPRAGE-like ADNI', 'Sagittal MPRAGE']
Index(['collection_id', 'image03_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_age', 'sex', 'comments_misc', 'image_file',
       'image_description', 'experiment_id', 'scan_type', 'scan_object',
       'image_file_format', 'image_modality', 'scanner_manufacturer_pd',
       'scanner_type_pd', 'scanner_software_versions_pd',
       'magnetic_field_strength', 'mri_repetition_time_pd', 'mri_echo_time_pd',
       'flip_angle', 'acquisition_matrix', 'mri_field_of_view_pd',
       'patient_position', 'photomet_interpret', 'transmit_coil',
       'transformation_performed', 'image_num_dimensions', 'image_extent1',
       'image_extent2', 'image_extent3', 'image_extent4', 'extent4_type',
       

src_subject_id  interview_age
1               252              2
1003            300              1
1005            624              1
1009            216              1
1011            588              1
                                ..
9986            744              3
S0623YIO        552              2
S2179ENY        240              2
S3430UFJ        0                2
S7976JLE        0                2
Length: 1093, dtype: int64

In [6]:
data_dir = os.path.join(path, 'image03')

#### CHANGE BASED ON WHAT 'image_file' COLUMN LOOKS LIKE IN image03 (corrects the file path to be on local machine) ####

# for 2274:
regex ='s3://NDAR_.*/submission_.[0-9]*/'


rows_with_bad_path = []
for index, row in image03.iterrows():
    image_path = re.sub(regex, '', row['image_file'])
    full_path = os.path.join(data_dir, image_path)
#     print(full_path)
    if not os.path.exists(full_path):
        print(image_path, index, row['image_file'])
        rows_with_bad_path.append(index)
rows_with_bad_path

[]

In [7]:
# drops rows based on rows with bad path and replaces old paths with local paths

image03 = image03.drop(rows_with_bad_path)
l = []

for index, row in image03.iterrows():
    image_path = re.sub(regex, '', row['image_file'])
    full_path = os.path.join(data_dir, image_path)
    l.append(full_path)
image03['image_file'] = l
image03

,collection_id,image03_id,dataset_id,subjectkey,src_subject_id,interview_age,image_file,image_description
827,2274,460912,11287,NDAR_INV11LD79PN,9715,372,../../physcosis/Package_1187332/image03/S4164MBS-1-3.zip,ADNI MPRAGE pg7 SENSE
829,2274,460914,11287,NDAR_INVWRL02K0R,8418,396,../../physcosis/Package_1187332/image03/S9096NNI-1-3.zip,ADNI MPRAGE pg7 SENSE
831,2274,460916,11287,NDAR_INV5V11KD2W,8871,372,../../physcosis/Package_1187332/image03/S0570LBU-1-3.zip,ADNI MPRAGE pg7 SENSE
833,2274,460918,11287,NDAR_INV7ACRD6KL,8041,576,../../physcosis/Package_1187332/image03/S5894LBP-1-3.zip,ADNI MPRAGE pg7 SENSE
835,2274,460920,11287,NDAR_INVRB1F3MW7,8530,276,../../physcosis/Package_1187332/image03/S1866HLF-1-3.zip,ADNI MPRAGE pg7 SENSE
...,...,...,...,...,...,...,...,...
9386,2274,461967,11293,NDAR_INVHUUMFYFA,272,456,../../physcosis/Package_1187332/image03/S0658KPN-1-2.zip,Sagittal MPRAGE-like ADNI
9388,2274,461969,11293,NDAR_INVDT0UDAHE,391,744,../../physcosis/Package_1187332/image03/S8519JOP-1-2.zip,Sagittal MPRAGE-like ADNI
9390,2274,461971,11293,NDAR_INV9C65L1XA,1409,336,../../physcosis/Package_1187332/image03/S9232RJB-1-2.zip,Sagittal MPRAGE-like ADNI
9392,2274,461973,11293,NDAR_INVRCC8D3DU,615,276,../../physcosis/Package_1187332/image03/S3383SOA-1-2.zip,Sagittal MPRAGE-like ADNI


In [8]:
### drop duplicates ###
print(image03.shape)
image03 = image03.drop_duplicates(subset = ['image_file', 'image_description'])
# image03 = image03.drop_duplicates(subset = ['src_subject_id', 'interview_age'])
print(image03.shape)

(1781, 8)
(1331, 8)


In [9]:
image03.pivot_table(columns = ['image_description'], aggfunc = 'size')

image_description
ADNI MPRAGE SENSE              15
ADNI MPRAGE pg7 SENSE         172
Bsnip Sag MPRAGE-like ADNI    291
MPRAGE                        170
MPRAGE Repeat                  33
MPRAGE_SAG                      3
MPRAGE_SAG Repeat               3
MPRAGE_repeat                  89
SAG MPRAGE                    107
Sagittal MPRAGE                 1
Sagittal MPRAGE-like ADNI      65
T1 MPRAGE                     339
T1 MPRAGE Repeat                5
WIP ADNI MPRAGE pg7 SENSE      38
dtype: int64

In [10]:
image03.pivot_table(columns = ['src_subject_id'], aggfunc = 'size')

src_subject_id
1           2
1003        1
1005        1
1009        1
1011        1
           ..
9986        1
S0623YIO    2
S2179ENY    2
S3430UFJ    2
S7976JLE    2
Length: 1092, dtype: int64

In [11]:
image03[image03['src_subject_id'] == 'S0623YIO']

,collection_id,image03_id,dataset_id,subjectkey,src_subject_id,interview_age,image_file,image_description
2269,2274,463300,11335,NDAR_INVAMUZFD5D,S0623YIO,552,../../physcosis/Package_1187332/image03/S0623YIO-1-2.zip,SAG MPRAGE
2270,2274,463301,11335,NDAR_INVAMUZFD5D,S0623YIO,552,../../physcosis/Package_1187332/image03/S0623YIO-1-3.zip,SAG MPRAGE


## Preprocess test

In [12]:
# count number of unique subject ids in the panss test 
print(test['src_subject_id'].nunique())
print(test.columns)
print(test.shape)
# I think only these are needed change is using another test
test = test[['src_subject_id', 'interview_age', 'panss_general',
       'panss_negative', 'panss_positive', 'pos_p1', 'pos_p2', 'pos_p3',
       'pos_p4', 'pos_p5', 'pos_p6', 'pos_p7', 'neg_n1', 'neg_n2', 'neg_n3',
       'neg_n4', 'neg_n5', 'neg_n6', 'neg_n7', 'gps_g1', 'gps_g2', 'gps_g3',
       'gps_g4', 'gps_g5', 'gps_g6', 'gps_g7', 'gps_g8', 'gps_g9', 'gps_g10',
       'gps_g11', 'gps_g12', 'gps_g13', 'gps_g14', 'gps_g15', 'gps_g16']]
test.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')


968
Index(['collection_id', 'panss01_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_age', 'sex', 'panss_general',
       'panss_negative', 'panss_positive', 'panss_total', 'pos_p1', 'pos_p2',
       'pos_p3', 'pos_p4', 'pos_p5', 'pos_p6', 'pos_p7', 'neg_n1', 'neg_n2',
       'neg_n3', 'neg_n4', 'neg_n5', 'neg_n6', 'neg_n7', 'gps_g1', 'gps_g2',
       'gps_g3', 'gps_g4', 'gps_g5', 'gps_g6', 'gps_g7', 'gps_g8', 'gps_g9',
       'gps_g10', 'gps_g11', 'gps_g12', 'gps_g13', 'gps_g14', 'gps_g15',
       'gps_g16', 'collection_title'],
      dtype='object')
(1779, 42)


src_subject_id                              interview_age                                                    
1                                           252                                                                  2
1005                                        624                                                                  2
1027                                        216                                                                  2
1033                                        312                                                                  2
1058                                        456                                                                  2
                                                                                                                ..
9894                                        384                                                                  3
9910                                        564                                      

In [13]:
# drop duplicate tests for subjects
test = test.drop_duplicates(subset = ['src_subject_id', 'interview_age'])
print(test.shape)

(968, 35)


## Preprocess ndar_subject

In [14]:
print(ndar_subject.shape)
ndar_subject.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')

(4404, 26)


src_subject_id                              interview_age                                                    
1                                           252                                                                  2
1003                                        300                                                                  2
1005                                        624                                                                  2
1009                                        216                                                                  2
1011                                        588                                                                  2
                                                                                                                ..
9986                                        744                                                                  4
999                                         348                                      

In [15]:
print(ndar_subject.shape)
ndar_subject = ndar_subject.drop_duplicates(subset = ['src_subject_id', 'interview_age'])
print(ndar_subject.shape)

(4404, 26)
(2416, 26)


## Join dataframes

In [16]:
s = 0
for i in ndar_subject[['src_subject_id', 'interview_age']].values.tolist():
    if i not in test[['src_subject_id', 'interview_age']].values.tolist():
        s = s + 1
print(s, " not in test but in ndar out of total in ndar ", ndar_subject.shape[0])



s = 0
for i in ndar_subject[['src_subject_id', 'interview_age']].values.tolist():
    if i not in image03[['src_subject_id', 'interview_age']].values.tolist():
        s = s + 1
print(s, " not in image03 but in ndar out of total in ndar ", ndar_subject.shape[0])




s = 0
for i in test[['src_subject_id', 'interview_age']].values.tolist():
    if i not in ndar_subject[['src_subject_id', 'interview_age']].values.tolist():
        s = s + 1
print(s, " not in ndar but in test out of total in test ", test.shape[0])



s = 0
for i in test[['src_subject_id', 'interview_age']].values.tolist():
    if i not in image03[['src_subject_id', 'interview_age']].values.tolist():
        s = s + 1
print(s, " not in image03 but in test out of total in test ", test.shape[0])



s = 0
for i in image03[['src_subject_id', 'interview_age']].values.tolist():
    if i not in test[['src_subject_id', 'interview_age']].values.tolist():
        s = s + 1
print(s, " not in test but in image03 out of total in image03 ", image03.shape[0])


s = 0
for i in image03[['src_subject_id', 'interview_age']].values.tolist():
    if i not in ndar_subject[['src_subject_id', 'interview_age']].values.tolist():
        s = s + 1
print(s, " not in ndar_subject but in image03 out of total in image03 ", image03.shape[0])


1448  not in test but in ndar out of total in ndar  2416
1369  not in image03 but in ndar out of total in ndar  2416
0  not in ndar but in test out of total in test  968
486  not in image03 but in test out of total in test  968
747  not in test but in image03 out of total in image03  1331
71  not in ndar_subject but in image03 out of total in image03  1331


In [17]:
res1 = image03.merge(ndar_subject, on = ['src_subject_id', 'interview_age'], how = 'inner')
print(res1.shape)
res2 = res1.merge(test,  on = ['src_subject_id', 'interview_age'], how = 'inner')


print(res2['src_subject_id'].nunique())
print(res2.columns)
print(res2.shape)

(1260, 32)
482
Index(['collection_id_x', 'image03_id', 'dataset_id_x', 'subjectkey_x',
       'src_subject_id', 'interview_age', 'image_file', 'image_description',
       'collection_id_y', 'ndar_subject01_id', 'dataset_id_y', 'subjectkey_y',
       'sex', 'race', 'phenotype', 'phenotype_description', 'twins_study',
       'sibling_study', 'family_study', 'family_user_def_id', 'src_mother_id',
       'src_father_id', 'src_sibling1_id', 'sibling_type1', 'src_sibling2_id',
       'sibling_type2', 'src_sibling3_id', 'sibling_type3', 'src_sibling4_id',
       'sibling_type4', 'sample_taken', 'collection_title', 'panss_general',
       'panss_negative', 'panss_positive', 'pos_p1', 'pos_p2', 'pos_p3',
       'pos_p4', 'pos_p5', 'pos_p6', 'pos_p7', 'neg_n1', 'neg_n2', 'neg_n3',
       'neg_n4', 'neg_n5', 'neg_n6', 'neg_n7', 'gps_g1', 'gps_g2', 'gps_g3',
       'gps_g4', 'gps_g5', 'gps_g6', 'gps_g7', 'gps_g8', 'gps_g9', 'gps_g10',
       'gps_g11', 'gps_g12', 'gps_g13', 'gps_g14', 'gps_g15', '

In [18]:
master_table = res2
master_table.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')

src_subject_id  interview_age
1               252              2
1005            624              1
1033            312              1
1058            456              1
1063            324              1
1082            264              1
110             324              1
1108            360              1
1115            336              1
1155            396              1
116             312              1
1226            312              1
1228            288              1
1236            588              1
1258            252              1
1260            408              1
1280            216              1
1294            396              1
1304            540              1
1336            612              1
1344            288              2
1347            456              1
135             432              1
1373            300              1
1383            216              1
1386            360              1
1396            252              1
1412            252      

In [19]:
master_table[master_table['src_subject_id'] == '1']

,collection_id_x,image03_id,dataset_id_x,subjectkey_x,src_subject_id,interview_age,image_file,image_description,collection_id_y,ndar_subject01_id,...,gps_g7,gps_g8,gps_g9,gps_g10,gps_g11,gps_g12,gps_g13,gps_g14,gps_g15,gps_g16
480,2274,462042,11293,NDAR_INVTJ6DFRKG,1,252,../../physcosis/Package_1187332/image03/S7644TYV-2-2.zip,MPRAGE,2274,323699,...,2,2,3,2,2,2,2,2,2,2
481,2274,462043,11293,NDAR_INVTJ6DFRKG,1,252,../../physcosis/Package_1187332/image03/S7644TYV-2-3.zip,MPRAGE Repeat,2274,323699,...,2,2,3,2,2,2,2,2,2,2


In [25]:
clean_dir = os.path.join(data_dir, 'CLEAN')

if not os.path.exists(clean_dir):
    os.mkdir(clean_dir)
clean_dir

'../../physcosis/Package_1187332/image03/CLEAN'

In [28]:
for i,row in master_table.iterrows():
    file_name = row['image_file'].split('/')[-1]
    folder = os.path.join(clean_dir,file_name.split('.')[0])
    if not os.path.exists(folder):
        os.system('mkdir ' + folder)
        os.system('unzip ' + row['image_file'] + ' -d ' + folder)
        print('Unziped and moved ', file_name)
        

In [29]:
print(len(os.listdir(clean_dir)))

584


In [36]:
for file in tqdm(os.listdir(clean_dir)):
    if os.path.isdir(os.path.join(clean_dir, file)):
        if not os.path.exists(os.path.join(clean_dir, file, '{}.nii'.format(file))):
            print('Converting', file, ' from dicom to niftii.')
            try:
                dicom2nifti.dicom_series_to_nifti(os.path.join(clean_dir,file), os.path.join(clean_dir, file,'{}.nii'.format(file)), reorient_nifti=True)
            except: 
                print(file, ' Failed.')

  0%|          | 0/584 [00:00<?, ?it/s]

../../physcosis/Package_1187332/image03/CLEAN/S9096NNI-1-3
Converting S9096NNI-1-3  from dicom to niftii.


  0%|          | 1/584 [00:00<08:26,  1.15it/s]

S9096NNI-1-3  Failed.
../../physcosis/Package_1187332/image03/CLEAN/S5894LBP-1-3
Converting S5894LBP-1-3  from dicom to niftii.


  0%|          | 2/584 [00:01<06:40,  1.45it/s]

...

100%|█████████▉| 582/584 [28:00<00:07,  3.66s/it]

../../physcosis/Package_1187332/image03/CLEAN/S0526FQI-1-3
Converting S0526FQI-1-3  from dicom to niftii.


100%|█████████▉| 583/584 [28:03<00:03,  3.57s/it]

../../physcosis/Package_1187332/image03/CLEAN/S1698NIH-1-3
Converting S1698NIH-1-3  from dicom to niftii.


100%|██████████| 584/584 [28:06<00:00,  2.89s/it]


In [ ]:
for file in tqdm(os.listdir(clean_dir)):
    if os.path.isdir(os.path.join(clean_dir, file)):
        if os.path.exists(os.path.join(clean_dir, file, '{}.nii'.format(file))):
            if not os.path.exists(os.path.join(clean_dir, file, file + '_defaced.nii')):                
                nifti_file = os.path.join(clean_dir, file, '{}.nii'.format(file))
                print('Defacing', nifti_file)
                os.system('pydeface ' + nifti_file)

  0%|          | 0/584 [00:00<?, ?it/s]

Defacing ../../physcosis/Package_1187332/image03/CLEAN/S7432UOB-1-2/S7432UOB-1-2.nii


 23%|██▎       | 133/584 [02:47<09:27,  1.26s/it]

Defacing ../../physcosis/Package_1187332/image03/CLEAN/S8240FWM-1-2/S8240FWM-1-2.nii
Defacing ../../physcosis/Package_1187332/image03/CLEAN/S5568DSN-1-2/S5568DSN-1-2.nii


 23%|██▎       | 135/584 [02:47<09:06,  1.22s/it]

...

Defacing ../../physcosis/Package_1187332/image03/CLEAN/S5331NGW-1-2/S5331NGW-1-2.nii


 29%|██▉       | 169/584 [2:16:59<27:03:35, 234.74s/it]

Defacing ../../physcosis/Package_1187332/image03/CLEAN/S5331NGW-1-5/S5331NGW-1-5.nii


 29%|██▉       | 170/584 [2:21:24<28:00:52, 243.60s/it]

Defacing ../../physcosis/Package_1187332/image03/CLEAN/S3311XPB-1-2/S3311XPB-1-2.nii
